In [ ]:
"""
Load the models
"""
from llm.api.openai_models import GPTChat
from llm.api.gemini_models import GeminiChat
from llm.api.azure_openai_models import AzureChat
from llm.api.deepinfra_models import DeepInfraChat
# mllm = DeepInfraChat("meta-llama/Llama-3.2-11B-Vision-Instruct")
# llm = AzureChat("gpt-4o-mini")

llm = GeminiChat("gemini-1.5-pro")
mllm = GeminiChat("gemini-1.5-pro")

"""
Load the dataset through customized json file
"""
from dataset import *
dataset = ImageDataset(llm)   # the model here used for "ENTITY EXTRACTION" !!!


In [ ]:
prefix = "gemini_"
file = "gemini_test"

In [ ]:
""" Entity Extraction """
dataset.load("load/mmvet.json")
dataset.entity_extraction(mllm, prefix + "1_entity_extraction")    # model used for "CAPTION GENERATION" !!!

In [ ]:
""" Question Decomposition """
dataset.load(f"save/{file}/{prefix}1_entity_extraction.json")
dataset.new_subquestion_generating(llm, prefix + "3_test_subquestion")

In [ ]:
"""
SubQuestion Answering (original image + cropped image)
"""
dataset.load(f"save/{file}/{prefix}3_test_subquestion.json")
dataset.new_subquestion_answering(mllm, prefix + "4_test_answer")

In [ ]:
"""
Summarization (Summarize the subquestion-answer pairs)
    - if no subquestion, just returns empty string info ""
"""
dataset.load(f"save/{file}/{prefix}4_test_answer.json")
dataset.new_summarization(llm, prefix + "5_test_summarize")


In [ ]:
"""
Final Question Answering
"""
dataset.load(f"save/{file}/{prefix}5_test_summarize.json")
dataset.question_answering(mllm, prefix + "6_test_answer")

In [ ]:
# from debator.base import Debate
# dataset.load(f"save/{file}/{prefix}6_test_answer.json")
# debator = Debate(llm=llm, mllm=mllm)
# dataset.debate(debator)
# dataset.save_json(f"{file}/{prefix}7_debater")

In [ ]:
import json

with open(f"save/{prefix}6_test_answer.json", "r") as f:
    data = json.load(f)

with open("results.json", "w") as output_f:
    results = {sol["id"]: sol["summarization"] for sol in data}
    json.dump(results, output_f, indent=4)